In [12]:
%%bash
pip install PyMuPDF
pip install tensorflow-addons
pip install gensim
pip install sent2vec
pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download en_core_web_trf

  Using cached spacy-3.5.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.1.10-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (924 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.7
    Uninstalling thinc-8.1.7:
      Successfully uninstalled thinc-8.1.7
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-trf 3.4.1 requires spacy<3.5.0,>=3.4.1, but you have spacy 3.5.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 32.1 MB/s eta 0:00:00
  Attempting uninstall: spacy-transformers
    Found existing installation: spacy-transformers 1.1.9
    Uninstalling spacy-transformers-1.1.9:
      Successfully uninstalled spacy-transformers-1.1.9
  Attempting uninstall: en-core-web-trf
    Found existing installation: en-core-web-trf 3.4.1
    Uninstalling en-core-web-trf-3.4.1:
      Successfully uninstalled en-core-web-trf-3.4.1


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


# Load libraries

In [1]:
import os
import re
import fitz
import random
from pathlib import Path 
from PIL import Image
import numpy as np
import pandas as pd
import spacy
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from transformers import (AutoModelForSequenceClassification,
                          TFAutoModelForSequenceClassification,
                          AutoTokenizer,
                          TFAutoModel,
                          TFDistilBertModel, 
                          TFDistilBertForSequenceClassification, 
                          DistilBertTokenizerFast, 
                          DistilBertConfig)

# Helper functions for Modeling
Some boilerplate to reduce rewriting code

In [2]:
def tokenize_sets(train, val, test, tokenizer, max_seq_len):
    """tokenize and encode sequences the set splits
    returns tokens dict from train, validation and test
    """
    tokens_train = tokenizer.batch_encode_plus(
        train.tolist(),
        padding =True,
        max_length = max_seq_len,
        truncation=True,
        return_token_type_ids=False
    )
    
    tokens_val = tokenizer.batch_encode_plus(
        val.tolist(),
        padding =True,
        max_length = max_seq_len,
        truncation=True,
        return_token_type_ids=False
    )

    tokens_test = tokenizer.batch_encode_plus(
        test.tolist(),
        padding =True,
        max_length = max_seq_len,
        truncation=True,
        return_token_type_ids=True
    )
    
    return tokens_train, tokens_val, tokens_test

def get_inputs(tokens_train, 
               train_labels, 
               tokens_val, 
               val_labels, 
               tokens_test, 
               test_labels):
    """Extracts input_ids and attention_mask 
    from tokenizer and turns them into numpy array
    """
    # for train set
    train_seq = np.array(tokens_train['input_ids'])
    train_mask = np.array(tokens_train['attention_mask'])
    train_y = np.array(train_labels)

    # for validation set
    val_seq = np.array(tokens_val['input_ids'])
    val_mask = np.array(tokens_val['attention_mask'])
    val_y = np.array(val_labels)

    # for test set
    test_seq = np.array(tokens_test['input_ids'])
    test_mask = np.array(tokens_test['attention_mask'])
    test_y = np.array(test_labels)
    
    return (train_seq, 
            train_mask, 
            train_y, 
            val_seq, 
            val_mask, 
            val_y, 
            test_seq, 
            test_mask, 
            test_y)

def evaluate(y_true, y_pred, report=True, f1_average="micro", target_names=None):
    """computes report and results from predicted labels
    """
    if report==True:
        print(f"{classification_report(y_true, y_pred, target_names=target_names)}\n")
        print("-"*20)
    print(f"f1-score: {f1_score(y_true, y_pred, average=f1_average):.4f}\n")

In [3]:
def e_greddy_remover(x, prob_remove, prob):
    """random remove class labels from inputs with probability prob
    """
    new = []
    xs =  x.split(" ")
    for t in xs:
        if t in prob_remove:
            if random.random() >= prob:
                new.append(t)
            else:
                pass
        else:
            new.append(t)
    return " ".join(new)

In [4]:
def prep_text(df, col, new_col):
    """ text processing function 
    """
    df[new_col] = df[col].apply(lambda x: x.lower())
    df[new_col] = df[new_col].apply(lambda x: re.sub(r'(\n+)(?=[a-zA-Z])', r' ', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub(r'(\n+)(?=[0-9]+)', r' ', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub(r'(\xa0+)(?=[a-zA-Z0-9])', r' ', x))
    df[new_col] = df[new_col].apply(lambda x: x.replace("\n"," \n "))
    df[new_col] = df[new_col].apply(lambda x: re.sub(' +', ' ', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub('•', '', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub('-.', '.', x))
    # df[new_col] = df[new_col].apply(lambda x: re.sub(r"[^a-zA-Z0-9 ]", r'', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub(r"\d+", r'', x))
    df[new_col] = df[new_col].apply(lambda x: re.sub(r"\s\s+", r' ', x))
    df[new_col] = df[new_col].apply(lambda x: x.rstrip().lstrip())
    return df


def remove_words_with_probs(df, col, new_col, remove_words=[], prob = 0.25, strict_remove=True):
    """ text processing function 
    """
    # delete remove words
    if remove_words != []:
        df[new_col] = df[col].apply(lambda x : e_greddy_remover(x, prob_remove=remove_words, prob=prob))
        
    if strict_remove:
        # make it unsensitive about others the majority class
        df[new_col] = df[new_col].apply(lambda x: " ".join([x for x in x.split(" ") if x not in ["other"]]))
    return df

# Preprocessing

# Chunking

# work imbalance

# Train Test Split

In [5]:
def split_sets(X,y, seed=123):
    train, temp, train_labels, temp_labels = train_test_split(X, y,
                                                          shuffle=True,
                                                          random_state=seed, 
                                                          test_size=0.3,
                                                          stratify=y)

    # we will use temp and temp to create validation and test set
    val, test, val_labels, test_labels = train_test_split(temp, 
                                                          temp_labels,
                                                          shuffle=True,
                                                          random_state=seed, 
                                                          test_size=0.5,
                                                          stratify=temp_labels)
    
    return train, train_labels, val, val_labels, test, test_labels

In [7]:
df_used = pd.read_csv("../data/oxml/augmented_chunks.csv", index_col=0)

In [35]:
df_used = df_used[(df_used["context_prep_len"]>=20) & (df_used["context_prep_len"]<=256)].copy()

In [37]:
cat_to_code = dict(zip(set(df_used["class"]), range(len(set(df_used["class"])))))
code_to_cat = dict(zip(range(len(set(df_used["class"]))), set(df_used["class"])))
print(cat_to_code)
print("-"*20)
print(code_to_cat)

{'social': 0, 'environmental': 1, 'governance': 2, 'other': 3}
--------------------
{0: 'social', 1: 'environmental', 2: 'governance', 3: 'other'}


In [38]:
X = df_used["context_prep"].copy()
y = df_used["class"].copy()

y = y.replace(cat_to_code)

In [39]:
train, train_labels, val, val_labels, test, test_labels = split_sets(X,y, seed=123)

In [41]:
base_tokenizer_prep = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokens_train, tokens_val, tokens_test = tokenize_sets(train, val, test, base_tokenizer_prep, max_seq_len=256)

In [42]:
train_seq, train_mask, train_y, val_seq, val_mask, val_y, test_seq, test_mask, test_y = get_inputs(tokens_train, 
                                                                                                   train_labels, 
                                                                                                   tokens_val, 
                                                                                                   val_labels, 
                                                                                                   tokens_test, 
                                                                                                   test_labels)

# Modeling

In [43]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

In [44]:
def predict_set(model, seq, mask):
    y_logits = model.predict([seq, mask])
    return np.argmax(y_logits, axis=1)

In [45]:
def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [46]:
def build_model( input_seq, input_mask,input_shape, model_name):
    transformer_layer = TFAutoModel.from_pretrained(model_name)

    
    main_layer = transformer_layer(input_seq, attention_mask=input_mask, output_hidden_states=True)
    
    h = layers.Concatenate()(main_layer.hidden_states)
    # h = layers.Dense(768, activation='relu',name = 'pre_classifier', 
    #                      kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02))(h)
    # h = layers.BatchNormalization()(main_layer.hidden_states[-1][:,0,:])
    h = layers.Flatten()(h)
    h = layers.BatchNormalization()(h)
    h = layers.Dropout(0.2, name="dropout")(h)
    out = layers.Dense(NUM_LABELS, activation='linear',name = "classifier", 
                       kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.02))(h)
    
    return tf.keras.Model(inputs=[input_seq,input_mask], outputs=out, name=model_name)

In [47]:
NUM_LABELS = len(set(y))

In [48]:
inp_shape = train_seq.shape[-1]
inps = layers.Input(shape = (inp_shape,), dtype='int32')
masks= layers.Input(shape = (inp_shape,), dtype='int32')

In [57]:
tf.keras.backend.clear_session()
model = build_model(input_seq = inps, 
                    input_mask = masks, 
                    input_shape=inp_shape, 
                    model_name = "distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['dropout_19', 'classifier', 'pre_classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [58]:
model.summary()

Model: "distilbert-base-uncased-finetuned-sst-2-english"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 256, 768),                                                   
                                 hidden_states=((No 

In [59]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
sample_weights = compute_sample_weight(class_weight="balanced",y=train_labels)
class_weights = compute_class_weight(class_weight="balanced",classes = np.unique(train_labels), y=train_labels)
class_weights = dict([x for x in zip(range(NUM_LABELS),class_weights)])

In [60]:
class_weights

{0: 1.2529122679286495,
 1: 1.6381485007139458,
 2: 2.1115030674846627,
 3: 0.4721841130470572}

In [61]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, 
              loss=loss,
              metrics=['accuracy', 
                       tfa.metrics.F1Score(num_classes=4, 
                                           average='micro',
                                            threshold=0.5)])

In [63]:
# train the model 
BATCH_SIZE = 24
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
history = model.fit([train_seq, train_mask],
                    train_y,
                    epochs = 1,
                    shuffle=True,
                    batch_size = BATCH_SIZE,
                    # class_weight = class_weights,
                    # sample_weight = sample_weights,
                    validation_data = ([val_seq, val_mask],val_y),
                    callbacks=[early_stopping])

574/574 [==============================] - 155s 264ms/step - loss: 4.8391 - accuracy: 0.8013 - f1_score: 0.6938 - val_loss: 5.6054 - val_accuracy: 0.7936 - val_f1_score: 0.6430


In [64]:
# y_pred_train = predict_set(model, train_seq, train_mask)
# y_pred_valid = predict_set(model, val_seq, val_mask)
y_pred_test = predict_set(model, test_seq, test_mask)

93/93 [==============================] - 10s 96ms/step


In [65]:
# evaluate(train_labels, y_pred_train, target_names=cat_to_code)
# print("-"*20)
# evaluate(val_labels, y_pred_valid, target_names=cat_to_code)
print("-"*20)
evaluate(test_labels, y_pred_test, target_names=cat_to_code)

--------------------
               precision    recall  f1-score   support

       social       0.73      0.65      0.69       589
environmental       0.71      0.74      0.72       451
   governance       0.56      0.68      0.62       349
        other       0.91      0.89      0.90      1562

     accuracy                           0.80      2951
    macro avg       0.73      0.74      0.73      2951
 weighted avg       0.80      0.80      0.80      2951


--------------------
f1-score: 0.7974



# Submission

In [66]:
def tokenize_single_set(tokenizer, data, max_seq_len):
    # tokenize and encode sequences in the validation set
    tokens = tokenizer.batch_encode_plus(
        data.tolist(),
        padding =True,
        max_length = max_seq_len,
        truncation=True,
        return_token_type_ids=False
    )
    return tokens

def get_single_inputs(tokens, labels):
    # returns tokens seq, masks and labels in numpy array for single set
    seq = np.array(tokens['input_ids'])
    mask = np.array(tokens['attention_mask'])
    label = np.array(labels)
    return seq, mask, label

In [67]:
dfsub = pd.read_csv("../data/oxml/submission_set.csv", index_col=0)

In [69]:
sub_labels_prep = prep_text(dfsub, col="context", new_col = "context_prep")
sub_tokens = tokenize_single_set(base_tokenizer_prep, 
                                     sub_labels_prep["context_prep"], 
                                     max_seq_len = 256)

sub_labels = sub_labels_prep["class"].replace(cat_to_code)
sub_seq, sub_mask, sub_y = get_single_inputs(sub_tokens, sub_labels)
y_pred =  predict_set(model, sub_seq, sub_mask)
dfsub["class"] = pd.Series(y_pred).replace(code_to_cat)

15/15 [==============================] - 1s 93ms/step


In [70]:
df_submission = dfsub[["id","class"]]

In [73]:
df_submission["class"].value_counts()

social           161
other            153
environmental     82
governance        70
Name: class, dtype: int64

In [74]:
df_submission.to_csv("../data/oxml/submission.csv", index=False)